# Inicializar bibliotecas

In [ ]:
!wget https://emanuel3queijos.github.io/AWS-University-ETL/python_scripts/requirements.txt -O requirements.txt
!sed -i '/pywin32/d' requirements.txt
!pip install -r requirements.txt



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Script de População do banco

In [ ]:
import psycopg2
from faker import Faker
import random

# Função para popular tabelas
def populate_tables():
# Tentando se conectar no banco da AWS
    try:
        conn = psycopg2.connect(host="db-manu-etluniversityproject-instance.c44vb5orqds7.us-east-1.rds.amazonaws.com",
                                dbname="AWS-University-ETL-DB",
                                user="postgres",
                                password="Emanu0710**",
                                port=5432)
        cur = conn.cursor()
        fake = Faker()

        # Definição dos departamentos
        departamentos = [
            ('Engenharia de Software', 'Departamento focado no desenvolvimento de software e tecnologias relacionadas.'),
            ('Ciência da Computação', 'Departamento que aborda a teoria da computação, algoritmos e inteligência artificial.'),
            ('Sistemas de Informação', 'Departamento que lida com sistemas computacionais e suas aplicações em organizações.'),
            ('Análise e Desenvolvimento de Sistemas', 'Departamento voltado para o desenvolvimento e análise de sistemas computacionais.'),
            ('Redes de Computadores', 'Departamento dedicado à infraestrutura e comunicação de redes de computadores.'),
            ('Inteligência Artificial', 'Departamento focado em IA, aprendizado de máquina e suas aplicações.'),
            ('Matemática Aplicada', 'Departamento com ênfase em técnicas matemáticas e suas aplicações em computação.')
        ]

        # Inserir departamentos
        for nome_departamento, descricao in departamentos:
            cur.execute("""
                INSERT INTO tb_departamento (nome_departamento, descricao)
                VALUES (%s, %s)
                """, (nome_departamento, descricao))

        # Definição dos cursos e descrições
        cursos = [
            ('Engenharia de Software', 'Curso sobre desenvolvimento de software.'),
            ('Ciência da Computação', 'Curso sobre fundamentos de computação.'),
            ('Sistemas de Informação', 'Curso sobre TI e gestão.'),
            ('Análise e Desenvolvimento de Sistemas', 'Curso sobre desenvolvimento de sistemas.'),
            ('Redes de Computadores', 'Curso sobre redes e comunicação.'),
            ('Inteligência Artificial', 'Curso sobre IA e aprendizado de máquina.')
        ]

        # Inserir cursos
        for nome_curso, descricao in cursos:
            cur.execute("""
                INSERT INTO tb_curso (nome, descricao)
                VALUES (%s, %s)
                """, (nome_curso, descricao))

        # Definição das disciplinas e créditos
        disciplinas = [
            ('Fundamentos de Programação', 5),
            ('Estruturas de Dados e Algoritmos', 4),
            ('Banco de Dados', 4),
            ('Engenharia de Requisitos', 3)
        ]

        # Inserir disciplinas
        for nome_disciplina, creditos in disciplinas:
            cur.execute("""
                INSERT INTO tb_disciplina (nome, creditos)
                VALUES (%s, %s)
                """, (nome_disciplina, creditos))

        # População de alunos e matrícula
        for _ in range(10000):  # Definindo a quantidade de alunos(rodei umas 3 vezes ent tem 30 mil no csv?)
            nome = fake.name()
            data_nascimento = fake.date_of_birth(minimum_age=18, maximum_age=40)
            email = fake.email()
            cur.execute("""
                INSERT INTO tb_aluno (nome, data_nascimento, email)
                VALUES (%s, %s, %s)
                RETURNING id_aluno
                """, (nome, data_nascimento, email))

            id_aluno = cur.fetchone()[0]
            id_curso = random.randint(1, len(cursos))
            data_matricula = fake.date_this_year()
            cur.execute("""
                INSERT INTO tb_matricula (id_aluno, id_curso, data_matricula)
                VALUES (%s, %s, %s)
                """, (id_aluno, id_curso, data_matricula))

            # Inscrição e turma
            id_disciplina = random.randint(1, len(disciplinas))
            data_inscricao = fake.date_this_year()
            cur.execute("""
                INSERT INTO tb_inscricao (id_aluno, id_disciplina, data_inscricao)
                VALUES (%s, %s, %s)
                """, (id_aluno, id_disciplina, data_inscricao))

        # Inserir turmas
        for id_disciplina in range(1, len(disciplinas) + 1):
            for ano in [2023, 2024]:
                for semestre in [1, 2]:
                    cur.execute("""
                        INSERT INTO tb_turma (id_disciplina, ano, semestre)
                        VALUES (%s, %s, %s)
                        """, (id_disciplina, ano, semestre))

        # Associar alunos a turmas
        for turma_id in range(1, 10):  # Para teste, associamos alunos em 10 turmas
            alunos_turma = random.sample(range(1, 101), random.randint(5, 10))
            for id_aluno in alunos_turma:
                status = random.choice(['Ativo', 'Concluído'])
                cur.execute("""
                    INSERT INTO tb_turma_aluno (id_turma, id_aluno, status)
                    VALUES (%s, %s, %s)
                    """, (turma_id, id_aluno, status))

        conn.commit()
        cur.close()
        print("Dados inseridos com sucesso.")
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Erro ao inserir dados: {error}")
    finally:
        if conn is not None:
            conn.close()



In [ ]:
# Criar as tabelas
create_tables()


Erro ao criar tabelas: ERRO:  relação "tb_aluno" já existe



In [ ]:
# Limpar as tabelas
def clean_tables():
    try:
        conn = psycopg2.connect(host="localhost", dbname="gestao_universitaria_ETL", user="postgres", password="0710", port=5432)
        cur = conn.cursor()
        truncate_sql = """TRUNCATE TABLE tb_turma_aluno, tb_inscricao, tb_matricula, tb_turma, tb_disciplina, tb_curso, tb_aluno RESTART IDENTITY CASCADE;"""
        cur.execute(truncate_sql)
        conn.commit()
        print("Tabelas limpas com sucesso")
        cur.close()

    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Erro ao truncar tabelas: {error}")

    finally:
        if conn is not None:
            conn.close()

clean_tables()

Tabelas limpas com sucesso


In [ ]:

def export_to_csv():
    try:
        conn = psycopg2.connect(host="localhost", dbname="gestao_universitaria_ETL", user="postgres", password="0710", port=5432)
        cur = conn.cursor()

        tables = ["tb_aluno", "tb_curso", "tb_disciplina", "tb_turma", "tb_matricula", "tb_inscricao", "tb_turma_aluno"]
        for table in tables:
            with open(f'{table}.csv', 'w') as f:
                cur.copy_expert(f"COPY {table} TO STDOUT WITH CSV HEADER", f)

        cur.close()
        print("Dados exportados com sucesso.")
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Erro ao exportar dados: {error}")
    finally:
        if conn is not None:
            conn.close()


#Export tables
export_to_csv()


Dados exportados com sucesso.
